In [1]:
import json
import numpy as np
import re
import utilities
from thai_tokenizer import Tokenizer 
f2 = open('sad_song.json', encoding="utf8")
f = open('songlist.json', encoding="utf8")

Data set preparing
- Extract Data from JSON
    - 5 Happy songs
    - 5 Sad songs

In [2]:
dict_happy = json.load(f)
f.close()
dict_sad = json.load(f2)
f2.close()

In [3]:
happy_song = []
sad_song = []
for i in dict_happy:
    happy_song.append(dict_happy[i])
for j in dict_sad:
    sad_song.append(dict_sad[j])


In [4]:
train_set = happy_song[:4] + sad_song[:4]
test_set = happy_song[4:] + sad_song[4:]
train_y = np.append(np.ones((len(happy_song[:4]), 1)), np.zeros((len(sad_song[:4]), 1)), axis=0)
test_y = np.append(np.ones((len(happy_song[4:]), 1)), np.zeros((len(sad_song[4:]), 1)), axis=0)
train_y2 = np.squeeze(train_y)
test_y2 = np.squeeze(test_y)
print(test_y2)

[1. 0.]


In [5]:
tokenizer = Tokenizer()

Model Class

In [6]:
class Is_this_a_sad_song_naive_bayes:
    def __init__(self,train_X, train_Y, test_X, test_Y) -> None:
        self.X = train_X
        self.Y = train_Y
        self.x = test_X
        self.y = test_y
        self.freqs = utilities.build_freq(train_y2,train_X)
        self.loglikelihood = {}
        self.logprior = 0
    def naive_bayes(self):
        vocab = set(x[0] for x in self.freqs)
        V = len(vocab)
        positive_cnts = 0
        negative_cnts = 0
        for pair in self.freqs.keys():
            if pair[1] >0:
                positive_cnts += self.freqs[pair]
            else:
                negative_cnts += self.freqs[pair]
        
        D = len(self.Y)
        positive_doc = list(self.Y).count(1)
        negative_doc = list(self.Y).count(0)
        self.logprior = np.log(positive_doc/negative_doc)
        for w in vocab:
            positive_freq = self.freqs.get((w, 1), 0)
            negative_freq = self.freqs.get((w, 0), 0)

            p_w_pos = (positive_freq + 1)/(positive_cnts + V)
            p_w_neg = (negative_freq + 1)/(negative_cnts + V)

            self.loglikelihood[w] = np.log(p_w_pos/p_w_neg)

    
    def predict(self,song,Process=utilities.Process):
        word_l = Process(song)
        p = 0
        p += self.logprior

        for word in word_l:
            if word in self.loglikelihood:
                p += self.loglikelihood[word]
        return p
    def test_naive_bayes(self):
        accuracy = 0  # return this properly

        y_hats = []
        for tweet in self.x:
            # if the prediction is > 0
            if self.predict(tweet) > 0:
                # the predicted class is 1
                y_hat_i = 1
            else:
                # otherwise the predicted class is 0
                y_hat_i = 0

            # append the predicted class to the list y_hats
            y_hats.append(y_hat_i)


        # error is the average of the absolute values of the differences between y_hats and test_y
        error = np.sum(abs(test_y2-y_hats))/len(self.y)
        # Accuracy is 1 minus the error
        accuracy = 1-error
        return accuracy
    
    def predict_result(self,song):
        p = self.predict(song)
        acc = self.test_naive_bayes()
        if p > 0:
            return print('Lyric:',song,' is not a sad song :D with accuracy = ',acc)
        else:
            return print('Lyric:',song,' is  a sad song ToT with accuracy = ',acc)
        
        

Building Model

In [7]:
MODEL = Is_this_a_sad_song_naive_bayes(train_set,train_y,test_set,test_y)
MODEL.naive_bayes() # train via naive bayes model

Predict result 

In [8]:
song1 = "ยินดีให้ความรักมันตาย ยินดีให้ชีวิตต้องพังทลาย"
MODEL.predict_result(song1)

Lyric: ยินดีให้ความรักมันตาย ยินดีให้ชีวิตต้องพังทลาย  is  a sad song ToT with accuracy =  1.0
